### Section 1: Setup
Point the script to location containing all of the data

In [1]:
import sys, os
import geopandas as gpd
import glob2
import os
import io
import zipfile
import pandas as pd
from shapely.geometry import Point, LineString, MultiLineString
from scipy import spatial
from functools import partial
import pyproj
from shapely.ops import transform
import re
from shapely.wkt import loads

In [2]:
data_location = r'C:\Users\charl\Documents\GOST\Vietnam\unzipped'

### Section 2: Diagnosis: Zipped Data
Here we try to identify what we are dealing with by iterating through all paths and subfolders, and summarizing the number of files / folders and their extensions

In [3]:
all_paths = glob2.glob(data_location+r'\**')
print('separate objects in folder tree: %s' % len(all_paths))

separate objects in folder tree: 25032


In [4]:
def PathDrDiagnose(path, verbose = False):
    
    print('for path %s:' % path)
    
    zipped_paths, csv_paths, kml_paths, rar_paths, others, file_extensions = [],[],[],[],[],[]
    
    all_paths = glob2.glob(path+r'\**')
    for z in all_paths:
        if z[-4:] == '.zip':
            zipped_paths.append(z)
        elif z[-4:] == '.csv':
            csv_paths.append(z)
        elif z[-4:] == '.kml':
            kml_paths.append(z)
        elif z[-4:] == '.rar':
            rar_paths.append(z)
        else:
            others.append(z)

        extension = os.path.splitext(z)[1]

        if len(extension) < 6:
                file_extensions.append(extension)

    df = pd.DataFrame({'extensions':file_extensions})

    print('zipped paths: %s' % len(zipped_paths))
    print('csv files: %s' % len(csv_paths))
    print('kml files: %s' % len(kml_paths))
    print('rar folders: %s' % len(rar_paths))
    if verbose == True:
        print(df.extensions.value_counts().to_frame())
    return rar_paths

In [5]:
rar_paths = PathDrDiagnose(data_location)

for path C:\Users\charl\Documents\GOST\Vietnam\unzipped:
zipped paths: 0
csv files: 10343
kml files: 8326
rar folders: 1


### Section 3: Unzipping
Generate new folders for all zipped paths and nested zipped paths (zipped folders inside zipped folders...)

In [6]:
def PathDrUnzip(in_path, unzip_path = None, delete = True, collapse=False, collapseDepth=5):
    
    if unzip_path == None:
        unzip_path = in_path
    
    error_counter = 0
    zipped_paths = []
    all_paths = glob2.glob(in_path+r'\**')
    
    for z in all_paths:
        if z[-4:] == '.zip':
            zipped_paths.append(z)
    
    def unzip(pth, in_path, unzip_path, delete = True):
        error_counter = 0
        subzip = []
        
        try:
            with zipfile.ZipFile(pth) as z:
                dirname = os.path.splitext(pth)[0]
                if os.path.exists(dirname):
                    pass
                else:
                    if collapse == True:
                        middle = os.path.relpath(pth, in_path)
                        middle = middle.split('\\')[:collapseDepth]
                        loc = os.path.join(unzip_path, *middle[:-1])
                    else:
                        loc = os.path.join(unzip_path, os.path.splitext(os.path.relpath(pth, in_path))[0])
                    if os.path.exists(loc):
                        print('passing - %s exists' % loc)
                        pass
                    else:
                        os.mkdir(loc)
                    for i in z.namelist():
                        if i[-4:] == '.zip':
                            print('adding subzip: ',os.path.join(loc, i))
                            subzip.append(os.path.join(loc, i))
                        print(i)
                        z.extract(i, loc)

            if delete == True: 
                os.unlink(pth)
        
        except:
            print('ERROR: cannot unzip %s' % pth)
            error_counter += 1
         
        return subzip, error_counter

    for zippy in zipped_paths:
        
        print('unzipping %s' % zippy)
        remnants, new_errors = unzip(zippy, in_path, unzip_path, delete)
        error_counter += new_errors
        
        while len(remnants) > 0:    
            
            for sub_zippy in remnants:
                
                print('unzipping sub-zip: %s' % sub_zippy)
                res, new_errors = unzip(sub_zippy, in_path, unzip_path, delete)
                remnants.remove(sub_zippy)
                error_counter += new_errors
                
                if len(res) > 0:
                    
                    print('further subfolders found')
                    remnants.append(*res)
                    
    print('errors encountered: %s' % error_counter)

### Section 4: Execute Unzipping

In [459]:
test_path = r'C:\Users\charl\Documents\GOST\Vietnam\unzipped'

print('pre unzip:')
PathDrDiagnose(test_path)

PathDrUnzip(test_path, test_path, delete = True, collapse=True, collapseDepth=2)

print('\npost unzip:')
PathDrDiagnose(test_path)

pre unzip:
for path C:\Users\charl\Documents\GOST\Vietnam\unzipped:
zipped paths: 401
csv files: 7678
kml files: 6051
rar folders: 2
unzipping C:\Users\charl\Documents\GOST\Vietnam\unzipped\Binh Dinh\Gửi DL GMTEF\Đường huyện\An Lão\313AL00001 (An Lão - An Vinh).zip
passing - C:\Users\charl\Documents\GOST\Vietnam\unzipped\Binh Dinh exists
/RoadLabPro/
RoadLabPro/2019_03_28_07_21/
RoadLabPro/2019_03_28_07_21/AL2019.3.22/
RoadLabPro/2019_03_28_07_21/AL2019.3.22/313AL00001/
RoadLabPro/2019_03_28_07_21/AL2019.3.22/313AL00001/313AL00001_Link_0001_2019_03_26_09_30/
RoadLabPro/2019_03_28_07_21/AL2019.3.22/313AL00001/313AL00001_Link_0001_2019_03_26_09_30/313AL00001_Link_0001_Bumps_2019_03_26_09_30.csv
RoadLabPro/2019_03_28_07_21/AL2019.3.22/313AL00001/313AL00001_Link_0001_2019_03_26_09_30/313AL00001_Link_0001_Path_2019_03_26_09_30.kml
RoadLabPro/2019_03_28_07_21/AL2019.3.22/313AL00001/313AL00001_Link_0001_2019_03_26_09_30/313AL00001_Link_0001_Path_2019_03_26_09_30.csv
RoadLabPro/2019_03_28_07_2

RoadLabPro/2019_04_05_13_40/AN2019.3.24/313AN00032/313AN00032_Link_0001_2019_03_29_13_51/313AN00032_Link_0001_Roughness_2019_03_29_13_51.kml
RoadLabPro/2019_04_05_13_40/AN2019.3.24/313AN00032/313AN00032_Link_0001_2019_03_29_13_51/313AN00032_Link_0001_Bumps_2019_03_29_13_51.kml
RoadLabPro/2019_04_05_13_40/AN2019.3.24/313AN00032/313AN00032_Link_0001_2019_03_29_13_51/313AN00032_Link_0001_Bumps_2019_03_29_13_51.csv
RoadLabPro/2019_04_05_13_40/AN2019.3.24/313AN00032/313AN00032_Link_0001_2019_03_29_13_51/313AN00032_Link_0001_Roughness_2019_03_29_13_51.csv
RoadLabPro/2019_04_05_13_40/AN2019.3.24/313AN00032/RoadSummary_2019_03_21_15_06_313AN00032.csv
unzipping C:\Users\charl\Documents\GOST\Vietnam\unzipped\Binh Dinh\Gửi DL GMTEF\Đường huyện\An Nhơn\313AN00033 Tân Dân - Bình Thạnh.zip
passing - C:\Users\charl\Documents\GOST\Vietnam\unzipped\Binh Dinh exists
/RoadLabPro/
RoadLabPro/2019_04_05_09_53/
RoadLabPro/2019_04_05_09_53/AN2019.3.24/
RoadLabPro/2019_04_05_09_53/AN2019.3.24/313AN00033/
Road

RoadLabPro/2019_03_25_10_09/AN2019.3.24/314AN00011/314AN00011_Link_0001_2019_03_22_09_40/314AN00011_Link_0001_Roughness_2019_03_22_09_40.csv
RoadLabPro/2019_03_25_10_09/AN2019.3.24/314AN00011/314AN00011_Link_0001_2019_03_22_09_40/314AN00011_Link_0001_Roughness_2019_03_22_09_40.kml
RoadLabPro/2019_03_25_10_09/AN2019.3.24/314AN00011/314AN00011_Link_0001_2019_03_22_09_40/314AN00011_Link_0001_Path_2019_03_22_09_40.kml
RoadLabPro/2019_03_25_10_09/AN2019.3.24/314AN00011/314AN00011_Link_0001_2019_03_22_09_40/314AN00011_Link_0001_Bumps_2019_03_22_09_40.csv
RoadLabPro/2019_03_25_10_09/AN2019.3.24/314AN00011/314AN00011_Link_0001_2019_03_22_09_40/314AN00011_Link_0001_Bumps_2019_03_22_09_40.kml
RoadLabPro/2019_03_25_10_09/AN2019.3.24/314AN00011/314AN00011_Link_0001_2019_03_22_09_40/314AN00011_Link_0001_Path_2019_03_22_09_40.csv
unzipping C:\Users\charl\Documents\GOST\Vietnam\unzipped\Binh Dinh\Gửi DL GMTEF\Đường huyện\Hoài Nhơn\313HN00007 TÀi Lương - Ka Công.zip
passing - C:\Users\charl\Documents\

passing - C:\Users\charl\Documents\GOST\Vietnam\unzipped\Binh Dinh exists
/RoadLabPro/
RoadLabPro/2019_04_08_10_31/
RoadLabPro/2019_04_08_10_31/Ho├ái_Nh╞ín_28.3.19/
RoadLabPro/2019_04_08_10_31/Ho├ái_Nh╞ín_28.3.19/313HN00013/
RoadLabPro/2019_04_08_10_31/Ho├ái_Nh╞ín_28.3.19/313HN00013/313HN00013_Link_0001_2019_03_28_09_54/
RoadLabPro/2019_04_08_10_31/Ho├ái_Nh╞ín_28.3.19/313HN00013/313HN00013_Link_0001_2019_03_28_09_54/313HN00013_Link_0001_Bumps_2019_03_28_09_54.csv
RoadLabPro/2019_04_08_10_31/Ho├ái_Nh╞ín_28.3.19/313HN00013/313HN00013_Link_0001_2019_03_28_09_54/313HN00013_Link_0001_Bumps_2019_03_28_09_54.kml
RoadLabPro/2019_04_08_10_31/Ho├ái_Nh╞ín_28.3.19/313HN00013/313HN00013_Link_0001_2019_03_28_09_54/313HN00013_Link_0001_Path_2019_03_28_09_54.csv
RoadLabPro/2019_04_08_10_31/Ho├ái_Nh╞ín_28.3.19/313HN00013/313HN00013_Link_0001_2019_03_28_09_54/313HN00013_Link_0001_Path_2019_03_28_09_54.kml
RoadLabPro/2019_04_08_10_31/Ho├ái_Nh╞ín_28.3.19/313HN00013/313HN00013_Link_0001_2019_03_28_09_54/31

passing - C:\Users\charl\Documents\GOST\Vietnam\unzipped\Binh Dinh exists
/RoadLabPro/
RoadLabPro/2019_04_08_10_58/
RoadLabPro/2019_04_08_10_58/Ho├ái_├én_27.3.19/
RoadLabPro/2019_04_08_10_58/Ho├ái_├én_27.3.19/313HA00020/
RoadLabPro/2019_04_08_10_58/Ho├ái_├én_27.3.19/313HA00020/313HA00020_Link_0001_2019_03_27_10_16/
RoadLabPro/2019_04_08_10_58/Ho├ái_├én_27.3.19/313HA00020/313HA00020_Link_0001_2019_03_27_10_16/313HA00020_Link_0001_Bumps_2019_03_27_10_16.csv
RoadLabPro/2019_04_08_10_58/Ho├ái_├én_27.3.19/313HA00020/313HA00020_Link_0001_2019_03_27_10_16/313HA00020_Link_0001_Bumps_2019_03_27_10_16.kml
RoadLabPro/2019_04_08_10_58/Ho├ái_├én_27.3.19/313HA00020/313HA00020_Link_0001_2019_03_27_10_16/313HA00020_Link_0001_Path_2019_03_27_10_16.csv
RoadLabPro/2019_04_08_10_58/Ho├ái_├én_27.3.19/313HA00020/313HA00020_Link_0001_2019_03_27_10_16/313HA00020_Link_0001_Path_2019_03_27_10_16.kml
RoadLabPro/2019_04_08_10_58/Ho├ái_├én_27.3.19/313HA00020/313HA00020_Link_0001_2019_03_27_10_16/313HA00020_Link_00

...

01.8.17/521b.2/L001-02-August-2017 18-41-52/Bumps_iPhone_5S_(GSM+CDMA)_20170802184152229.csv
01.8.17/521b.2/L001-02-August-2017 18-41-52/Bumps_iPhone_5S_(GSM+CDMA)_20170802184152229.kml
01.8.17/521b.2/L001-02-August-2017 18-41-52/Intervals_iPhone_5S_(GSM+CDMA)_20170802184152229.csv
01.8.17/521b.2/L001-02-August-2017 18-41-52/Intervals_iPhone_5S_(GSM+CDMA)_20170802184152229.kml
01.8.17/521b.2/L001-02-August-2017 18-41-52/RoadPath_iPhone_5S_(GSM+CDMA)_20170802184152229-gps.csv
01.8.17/521b.2/L001-02-August-2017 18-41-52/RoadPath_iPhone_5S_(GSM+CDMA)_20170802184152229-gps.kml
01.8.17/521c/
01.8.17/521c/L001-02-August-2017 16-14-10/
01.8.17/521c/L001-02-August-2017 16-14-10/Bumps_iPhone_5S_(GSM+CDMA)_20170802161410211.csv
01.8.17/521c/L001-02-August-2017 16-14-10/Bumps_iPhone_5S_(GSM+CDMA)_20170802161410211.kml
01.8.17/521c/L001-02-August-2017 16-14-10/Intervals_iPhone_5S_(GSM+CDMA)_20170802161410211.csv
01.8.17/521c/L001-02-August-2017 16-14-10/Intervals_iPhone_5S_(GSM+CDMA)_2017080216141

14.7.17/526.1/L002-14-July-2017 09-46-27/RoadPath_iPhone_5S_(GSM+CDMA)_20170714094627477-gps.kml
14.7.17/526.2/
14.7.17/527/
14.7.17/527.b/
14.7.17/527.b/L001-14-July-2017 13-06-01/
14.7.17/527.b/L001-14-July-2017 13-06-01/Bumps_iPhone_5S_(GSM+CDMA)_20170714130601229.csv
14.7.17/527.b/L001-14-July-2017 13-06-01/Bumps_iPhone_5S_(GSM+CDMA)_20170714130601229.kml
14.7.17/527.b/L001-14-July-2017 13-06-01/Intervals_iPhone_5S_(GSM+CDMA)_20170714130601229.csv
14.7.17/527.b/L001-14-July-2017 13-06-01/Intervals_iPhone_5S_(GSM+CDMA)_20170714130601229.kml
14.7.17/527.b/L001-14-July-2017 13-06-01/RoadPath_iPhone_5S_(GSM+CDMA)_20170714130601229-gps.csv
14.7.17/527.b/L001-14-July-2017 13-06-01/RoadPath_iPhone_5S_(GSM+CDMA)_20170714130601229-gps.kml
14.7.17/527c/
14.7.17/527c/L002-14-July-2017 13-24-21/
14.7.17/527c/L002-14-July-2017 13-24-21/Bumps_iPhone_5S_(GSM+CDMA)_20170714132421343.csv
14.7.17/527c/L002-14-July-2017 13-24-21/Bumps_iPhone_5S_(GSM+CDMA)_20170714132421343.kml
14.7.17/527c/L002-14-Ju

19.7.2017/502.1/L001-19-July-2017 08-28-17/Intervals_iPhone_5S_(GSM+CDMA)_20170719082817229.kml
19.7.2017/502.1/L001-19-July-2017 08-28-17/RoadPath_iPhone_5S_(GSM+CDMA)_20170719082817229-gps.csv
19.7.2017/502.1/L001-19-July-2017 08-28-17/RoadPath_iPhone_5S_(GSM+CDMA)_20170719082817229-gps.kml
19.7.2017/515/
19.7.2017/515/L001-19-July-2017 08-55-00/
19.7.2017/515/L001-19-July-2017 08-55-00/Bumps_iPhone_5S_(GSM+CDMA)_20170719085500197.csv
19.7.2017/515/L001-19-July-2017 08-55-00/Bumps_iPhone_5S_(GSM+CDMA)_20170719085500197.kml
19.7.2017/515/L001-19-July-2017 08-55-00/Intervals_iPhone_5S_(GSM+CDMA)_20170719085500197.csv
19.7.2017/515/L001-19-July-2017 08-55-00/Intervals_iPhone_5S_(GSM+CDMA)_20170719085500197.kml
19.7.2017/515/L001-19-July-2017 08-55-00/RoadPath_iPhone_5S_(GSM+CDMA)_20170719085500197-gps.csv
19.7.2017/515/L001-19-July-2017 08-55-00/RoadPath_iPhone_5S_(GSM+CDMA)_20170719085500197-gps.kml
19.7.2017/515C/
19.7.2017/515C/L001-19-July-2017 09-56-03/
19.7.2017/515C/L001-19-July-

24.7.17/212TH00054/L001-24-July-2017 16-14-54/Intervals_iPhone_5S_(GSM+CDMA)_20170724161454449.csv
24.7.17/212TH00054/L001-24-July-2017 16-14-54/Intervals_iPhone_5S_(GSM+CDMA)_20170724161454449.kml
24.7.17/212TH00054/L001-24-July-2017 16-14-54/RoadPath_iPhone_5S_(GSM+CDMA)_20170724161454449-gps.csv
24.7.17/212TH00054/L001-24-July-2017 16-14-54/RoadPath_iPhone_5S_(GSM+CDMA)_20170724161454449-gps.kml
24.7.17/212TH00054/L001-24-July-2017 16-56-33/
24.7.17/212TH00054/L001-24-July-2017 16-56-33/Bumps_iPhone_5S_(GSM+CDMA)_20170724165633231.csv
24.7.17/212TH00054/L001-24-July-2017 16-56-33/Bumps_iPhone_5S_(GSM+CDMA)_20170724165633231.kml
24.7.17/212TH00054/L001-24-July-2017 16-56-33/Intervals_iPhone_5S_(GSM+CDMA)_20170724165633231.csv
24.7.17/212TH00054/L001-24-July-2017 16-56-33/Intervals_iPhone_5S_(GSM+CDMA)_20170724165633231.kml
24.7.17/212TH00054/L001-24-July-2017 16-56-33/RoadPath_iPhone_5S_(GSM+CDMA)_20170724165633231-gps.csv
24.7.17/212TH00054/L001-24-July-2017 16-56-33/RoadPath_iPhone

zipped paths: 0
csv files: 10159
kml files: 8179
rar folders: 2


### Section 5: Diagnosis: Unzipped Data

In [7]:
road_summaries, intervals, roadpaths, roughness, paths = [], [], [], [], []

path = data_location
all_paths = glob2.glob(path+r'\**')
for z in all_paths:
    if 'RoadSummary' in z and '.csv' in z:
        road_summaries.append(z)
    elif '_Roughness_' in z and '.csv' in z:
        roughness.append(z)
    elif '_Path_' in z and '.csv' in z:
        paths.append(z)
    elif 'Intervals_' in z and '.csv' in z:
        intervals.append(z)
    elif 'RoadPath_' in z and '.csv' in z:
        roadpaths.append(z)

print('total number of csvs with "RoadSummary" in the name: %d' % len(road_summaries))
print('total number of csvs with "_Path_" in the name: %d' % len(paths))
print('total number of csvs with "_Roughness_" in the name: %d' % len(roughness))
print('total number of csvs with "_Intervals_" in the name: %d' % len(intervals))
print('total number of csvs with "_RoadPath_" in the name: %d' % len(roadpaths))

total number of csvs with "RoadSummary" in the name: 1815
total number of csvs with "_Path_" in the name: 1994
total number of csvs with "_Roughness_" in the name: 1995
total number of csvs with "_Intervals_" in the name: 481
total number of csvs with "_RoadPath_" in the name: 481


In [8]:
# New Style Traces
new_style_RLP_roots = []
for fil in roughness:
    root = os.path.join(*fil.split('\\')[:-1])
    subs = glob2.glob(root.replace('C:Users',r'C:\Users')+r'\**')
    counter = 0
    for sub in subs:
        if '_Roughness_' in sub and '.csv' in sub:
            counter = 1
    if counter == 1:
        new_style_RLP_roots.append(root)

valid_new_traces = []
for new in new_style_RLP_roots:
    subs = glob2.glob(new.replace('C:Users',r'C:\Users')+r'\**')
    intervals_counter, roadpaths_counter = 0, 0
    for sub in subs:
        if '_Roughness_' in sub and '.csv' in sub:
            interval_fil = sub
            intervals_counter = 1
        elif '_Path_' in sub and '.csv' in sub:
            roadpath_fil = sub
            roadpaths_counter = 1
    if intervals_counter == 1 and roadpaths_counter == 1:
        valid_new_traces.append([roadpath_fil, interval_fil])

# Old Style Traces
old_style_RLP_roots = []
for fil in intervals:
    root = os.path.join(*fil.split('\\')[:-1])
    subs = glob2.glob(root.replace('C:Users',r'C:\Users')+r'\**')
    counter = 0
    for sub in subs:
        if 'Intervals' in sub and '.csv' in sub:
            counter = 1
    if counter == 1:
        old_style_RLP_roots.append(root)

valid_old_traces = []
for old in old_style_RLP_roots:
    subs = glob2.glob(old.replace('C:Users',r'C:\Users')+r'\**')
    intervals_counter, roadpaths_counter = 0, 0
    for sub in subs:
        if 'Intervals' in sub and '.csv' in sub:
            interval_fil = sub
            intervals_counter = 1
        elif 'RoadPath' in sub and '.csv' in sub:
            roadpath_fil = sub
            roadpaths_counter = 1
    if intervals_counter == 1 and roadpaths_counter == 1:
        valid_old_traces.append([roadpath_fil, interval_fil])

print('Valid New-Style Data: %s' % len(valid_new_traces))
print('Valid Old-Style Data: %s' % len(valid_old_traces))
print('Total valid traces: %d' % (len(valid_old_traces) + len(valid_new_traces)))

Valid New-Style Data: 1994
Valid Old-Style Data: 481
Total valid traces: 2475


### Section 6: Defining Class Objects

In [9]:
class NewTrace(object):
    
    def __init__(self, path_list):
        self.path = path_list[0]
        self.roughness = path_list[1]
        
    def shittyPandasOpen(self, fileName):
            '''https://stackoverflow.com/questions/1365797/python-long-filename-support-broken-in-windows
            '''
            allResults = []
            lineCnt = 0
            fileName = r"\\?\%s" % fileName
            with open(fileName, 'r', encoding="utf8") as inFile:
                for line in inFile:
                    if lineCnt == 0:
                        titles = line.split('","')
                        titles[-1] = titles[-1].replace('\n','')
                        for t in range(0, len(titles)):
                            titles[t] = titles[t].replace('"','')
                    else:
                        data = line.split('","')
                        data[-1] = data[-1].replace(',\n','')
                        for t in range(0, len(data)):
                            data[t] = data[t].replace('"','')
                        allResults.append(data)
                    lineCnt = lineCnt + 1

            if len(allResults) > 0:
                xx = pd.DataFrame(allResults, columns=titles)
            else:  
                raise ValueError('Error: file has no length')

            return(xx)
        

    def load(self):
        
        ### Path file
        self.path_df = self.shittyPandasOpen(self.path) 
        self.path_df.columns = [x.upper() for x in self.path_df.columns]
        if self.path_df.columns[3] == 'POINT_LATIDUDE':
            inColumns = list(self.path_df.columns)
            inColumns[3] = 'POINT_LATITUDE'
            self.path_df.columns = inColumns
        self.path_df['Points'] = list(zip(self.path_df.POINT_LONGITUDE.astype(float), 
                                          self.path_df.POINT_LATITUDE.astype(float)))
        self.path_df['Points'] = self.path_df['Points'].apply(Point)
        if len(self.path_df) < 2:
            raise ValueError('Error: Path DataFrame contains one or fewer entries - cannot generate a line!')
            self.path_geometry = None
        else:
            self.path_geometry = LineString(list(self.path_df.Points))
        
        ### Roughness file
        self.roughness_df = self.shittyPandasOpen(self.roughness) 
        self.roughness_df.columns = [x.upper() for x in self.roughness_df.columns]
        try:
            if self.path_df.ROAD_IDENTIFICATION.iloc[0] == self.roughness_df.ROAD_IDENTIFICATION.iloc[0]:
                self.road_ID = self.path_df.ROAD_IDENTIFICATION.iloc[0]
            else:
                raise ValueError('Error: Path and Roughness files return different IDs!!')
        except:
            raise ValueError('Error: no "ROAD_IDENTIFICATION" field in DF - file corrupted')

    def checkGeometry(self, 
                      source = 'epsg:4326', 
                      target = 'epsg:3405', 
                      thresh = 300, 
                      min_coords = 4,
                      min_length = 200):
        
        if self.path_geometry == None:
            raise ValueError('Error: Cannot check geometry - No geometry to check!')
        else:
        
            self.new_geometry = self.path_geometry

            broken_roads = []
            breaks = []

            project_WGS_UTM = partial(
                    pyproj.transform,
                    pyproj.Proj(init=source),
                    pyproj.Proj(init=target))

            project_UTM_WGS = partial(
                        pyproj.transform,
                        pyproj.Proj(init=target),
                        pyproj.Proj(init=source))

            coord_list = self.path_geometry.coords

            for i in range(1, (len(coord_list))):
                st_coord = coord_list[i-1]
                end_coord = coord_list[i]
                segment = LineString([Point(st_coord),Point(end_coord)])
                proj_seg = transform(project_WGS_UTM, segment)

                if proj_seg.length > thresh:
                    breaks.append(i)

            if len(breaks) == 0:
                pass

            else:
                new_geoms = []
                breaks.append(len(coord_list))
                st = 0
                for i in breaks:
                    coord_seq = coord_list[st:i]
                    if len(coord_seq) > min_coords: 
                        new_feature = LineString(coord_seq)
                        new_feature_UTM = transform(project_WGS_UTM, new_feature)
                        if new_feature_UTM.length > min_length:
                            new_geoms.append(LineString(coord_seq))
                    st = i

                if len(new_geoms) == 1:
                    self.new_geometry = new_geoms[0]

                elif len(new_geoms) > 1:
                    self.new_geometry = MultiLineString(new_geoms)

                else:
                    print('invalid geometry! (%s) geoms' % len(new_geoms),self.road_ID,self.path)
                    self.new_geometry = None
    
    def conditionSummary(self,mapper):
        
        self.suspension = self.roughness_df.SUSPENSION_TYPE.iloc[0]
        
        cut = self.roughness_df.copy()
        cut.CONDITION_CATEGORY = cut.CONDITION_CATEGORY.apply(lambda x: x.upper())
        cut = cut.loc[cut.CONDITION_CATEGORY != 'NOT MEASURED']
        cut.CONDITION_CATEGORY = cut.CONDITION_CATEGORY.map(mapper)
        cut.INTERVAL_LENGTH = cut.INTERVAL_LENGTH.astype(float)
        total_interval_length = cut.INTERVAL_LENGTH.sum()
        cut['WEIGHTED'] = cut.INTERVAL_LENGTH * cut.CONDITION_CATEGORY
        average_state = cut.WEIGHTED.sum() / total_interval_length
        self.condition = (round(average_state))
        
        cut = self.roughness_df.copy()
        cut = cut.loc[cut.ROUGHNESS != '']
        cut.INTERVAL_LENGTH = cut.INTERVAL_LENGTH.astype(float)
        cut.ROUGHNESS = cut.ROUGHNESS.astype(float)
        cut['WEIGHTED_IRI'] = cut.INTERVAL_LENGTH * cut.ROUGHNESS
        average_state = cut.WEIGHTED_IRI.sum() / total_interval_length
        self.IRI_average = average_state

In [10]:
class OldTrace(object):
    
    def __init__(self, path_list):
        self.path = path_list[0]
        self.roughness = path_list[1]
        
    def shittyPandasOpen(self, fileName):
        '''https://stackoverflow.com/questions/1365797/python-long-filename-support-broken-in-windows
        '''
        allResults = []
        lineCnt = 0
        fileName = r"\\?\%s" % fileName
        with open(fileName, 'r', encoding="utf8") as inFile:
            for line in inFile:
                line = line.replace('"', '')
                if lineCnt == 0:
                    titles = line.split(",")
                    titles[-1] = titles[-1].replace('\n','')
                else:
                    data_line = line.split(",")
                    data_line[-1] = data_line[-1].replace('\n','')
                    allResults.append(data_line)
                lineCnt = lineCnt + 1
        
        if len(allResults) > 0:
            xx = pd.DataFrame(allResults, columns=titles[:len(allResults[0])])
        else:  
            raise ValueError('Error: file has no length')
            
        return(xx)

    def load(self):
        
        ### Path file
        self.path_df = self.shittyPandasOpen(self.path) 
        self.path_df.columns = [x.upper() for x in self.path_df.columns]
        self.path_df['Points'] = list(zip(self.path_df.LONGITUDE.astype(float), 
                                          self.path_df.LATITUDE.astype(float)))
        self.path_df['Points'] = self.path_df['Points'].apply(Point)
        if len(self.path_df) < 2:
            raise ValueError('Error: Path DataFrame contains one or fewer entries - cannot generate a line!')
            self.path_geometry = None
        else:
            self.path_geometry = LineString(list(self.path_df.Points))
        
        ### Roughness file
        self.roughness_df = self.shittyPandasOpen(self.roughness) 
        self.roughness_df.columns = [x.upper() for x in self.roughness_df.columns]
    
    def checkGeometry(self, 
                      source = 'epsg:4326', 
                      target = 'epsg:3405', 
                      thresh = 300, 
                      min_coords = 4, 
                      min_length = 200):
        
        if self.path_geometry == None:
            raise ValueError('Error: Cannot check geometry - No geometry to check!')
        else:
            self.new_geometry = self.path_geometry
        
            broken_roads = []
            breaks = []

            project_WGS_UTM = partial(
                    pyproj.transform,
                    pyproj.Proj(init=source),
                    pyproj.Proj(init=target))

            project_UTM_WGS = partial(
                        pyproj.transform,
                        pyproj.Proj(init=target),
                        pyproj.Proj(init=source))

            coord_list = self.path_geometry.coords

            for i in range(1, (len(coord_list))):
                st_coord = coord_list[i-1]
                end_coord = coord_list[i]
                segment = LineString([Point(st_coord),Point(end_coord)])
                proj_seg = transform(project_WGS_UTM, segment)

                if proj_seg.length > thresh:
                    breaks.append(i)

            if len(breaks) == 0:
                pass

            else:
                new_geoms = []
                breaks.append(len(coord_list))
                st = 0
                for i in breaks:
                    coord_seq = coord_list[st:i]
                    if len(coord_seq) > min_coords: 
                        new_feature = LineString(coord_seq)
                        new_feature_UTM = transform(project_WGS_UTM, new_feature)
                        if new_feature_UTM.length > min_length:
                            new_geoms.append(LineString(coord_seq))
                    st = i

                if len(new_geoms) == 1:
                    self.new_geometry = new_geoms[0]

                elif len(new_geoms) > 1:
                    self.new_geometry = MultiLineString(new_geoms)

                else:
                    print('invalid geometry! (%s) geoms' % len(new_geoms),self.road_ID,self.path)
                    self.new_geometry = None
    
    def conditionSummary(self,mapper):
                
        self.roughness_df.CATEGORY = self.roughness_df.CATEGORY.map(mapper)
        self.roughness_df.DISTANCE = self.roughness_df.DISTANCE.astype(float)
        total_interval_length = self.roughness_df.DISTANCE.sum()
        self.roughness_df['WEIGHTED_CAT'] = self.roughness_df.DISTANCE * self.roughness_df.CATEGORY
        average_state = self.roughness_df.WEIGHTED_CAT.sum() / total_interval_length
        self.condition = (round(average_state))
        
        self.roughness_df.IRI = self.roughness_df.IRI.astype(float)
        self.roughness_df['WEIGHTED_IRI'] = self.roughness_df.DISTANCE * self.roughness_df.IRI
        average_state = self.roughness_df.WEIGHTED_IRI.sum() / total_interval_length
        self.IRI_average = (average_state)
        self.suspension = self.roughness_df.SUSPENSION.iloc[0]
    
    def searchPathforVPROMMS(self):
        search_res = re.search(r"[1-9][1-4][1-4].*\\", self.path)
        if search_res == None:
            base_pth = r'C:\\Users\\charl\\Documents\\GOST\\Vietnam\\unzipped'
            self.road_ID = os.path.relpath(self.path, base_pth) 
        else:
            self.road_ID = search_res.group().split('\\')[0]

### Section 7: Execution

In [38]:
mapper = {'NOT MEASURED':None,
         'VERY POOR':0,
         'POOR':1,
         'FAIR':2.0,
         'GOOD':3.0,
         'VERY GOOD':4.0,
         'EXCELLENT':5.0}

error_paths = []
base_pth = r'C:\\Users\\charl\\Documents\\GOST\\Vietnam\\unzipped'

error_counter = 0
processed_new_traces = []
val = 0
verbose = 0
thresh = 300 # metres beyond which to declare points erroneous
min_coords = 3 # minimum coordinate number for a new geometry object
min_length = 150 # minimum length, in metres, for a new geometry object

print('New model: Commencing trace processing')
for trace_candidate in valid_new_traces:
    try:
        trace = NewTrace(trace_candidate)
        trace.load()
        trace.checkGeometry(thresh = thresh, min_coords = min_coords, min_length = min_length)
        trace.conditionSummary(mapper)
        processed_new_traces.append(trace)
    except ValueError as err:
        if verbose == 1:
            print("Value Error: %s, %s, %s" % (err, val, trace.path))
        error_counter+=1
        error_paths.append([os.path.relpath(trace_candidate[0], base_pth),err])
    except AttributeError as err:
        if verbose == 1:
            print("Attribute Error: %s, %s, %s" % (err, val, trace.path))
        error_counter+=1
        error_paths.append([os.path.relpath(trace_candidate[0], base_pth),err])
    except AssertionError as err:
        if verbose == 1:
            print("Weird column error ... probably: %s, %s, %s" % (err, val, trace.path))
        error_counter+=1
        error_paths.append([os.path.relpath(trace_candidate[0], base_pth),err])
    val+=1
        
print('\nNew model: traces generated: %s' % len(processed_new_traces))
print('New model: errors encountered: %s' % error_counter)

error_counter = 0
processed_old_traces = []
print('\nOld model: Commencing trace processing')
for trace_candidate in valid_old_traces:
    try:
        trace = OldTrace(trace_candidate)
        trace.load()
        trace.checkGeometry(thresh = thresh, min_coords = min_coords, min_length = min_length)
        trace.searchPathforVPROMMS()
        trace.conditionSummary(mapper)
        processed_old_traces.append(trace)
    except:
        error_counter+=1
        error_paths.append([os.path.relpath(trace_candidate[0], base_pth),'unknown'])
        pass

print('\nOld model: traces generated: %s' % len(processed_old_traces))
print('Old model: errors encountered: %s' % error_counter)

processing_errors = pd.DataFrame(error_paths, columns = ['FilePath','Error'])
processing_errors['road_ID'] = 'trace never formed - cannot determine road_ID'

New model: Commencing trace processing
invalid geometry! (0) geoms 034BQ00024 C:\Users\charl\Documents\GOST\Vietnam\unzipped\Ha Giang\WBank\WBank\Huyện Bắc Quang\034BQ00024\034BQ00024_Link_0001_2019_03_01_09_41\034BQ00024_Link_0001_Path_2019_03_01_09_41.csv


C:\Users\charl\Anaconda3\envs\Cfox2\lib\site-packages\ipykernel_launcher.py:139: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\charl\Anaconda3\envs\Cfox2\lib\site-packages\ipykernel_launcher.py:147: RuntimeWarning: invalid value encountered in double_scalars


invalid geometry! (0) geoms 034BQ00225 C:\Users\charl\Documents\GOST\Vietnam\unzipped\Ha Giang\WBank\WBank\Huyện Bắc Quang\034BQ00225\034BQ00225_Link_0001_2019_03_26_09_30\034BQ00225_Link_0001_Path_2019_03_26_09_30.csv
invalid geometry! (0) geoms 034BQ00227 C:\Users\charl\Documents\GOST\Vietnam\unzipped\Ha Giang\WBank\WBank\Huyện Bắc Quang\034BQ00227\034BQ00227_Link_0001_2019_03_26_09_26\034BQ00227_Link_0001_Path_2019_03_26_09_26.csv
invalid geometry! (0) geoms 034BQ00228 C:\Users\charl\Documents\GOST\Vietnam\unzipped\Ha Giang\WBank\WBank\Huyện Bắc Quang\034BQ00228\034BQ00228_Link_0001_2019_03_26_09_22\034BQ00228_Link_0001_Path_2019_03_26_09_22.csv
invalid geometry! (0) geoms 034BQ00277 C:\Users\charl\Documents\GOST\Vietnam\unzipped\Ha Giang\WBank\WBank\Huyện Bắc Quang\034BQ00277\034BQ00277_Link_0001_2019_03_26_16_31\034BQ00277_Link_0001_Path_2019_03_26_16_31.csv
invalid geometry! (0) geoms 034BQ00375 C:\Users\charl\Documents\GOST\Vietnam\unzipped\Ha Giang\WBank\WBank\Huyện Bắc Quang\0

### Section 8: Form DataFrame of Processed Traces

In [41]:
rows = []
for trace in processed_new_traces:
    rows.append({
        'province':os.path.relpath(trace.path, base_pth).split('\\')[0],
        'road_ID':trace.road_ID,
        'geometry':trace.new_geometry,
        'condition':trace.condition,
        'IRI_average':trace.IRI_average, 
        'suspension':trace.suspension,
        'trace_type':'new_trace',
        'FilePath':os.path.relpath(trace.path, base_pth)
    })
for trace in processed_old_traces:
    rows.append({
        'province':os.path.relpath(trace.path, base_pth).split('\\')[0],
        'road_ID':trace.road_ID,
        'geometry':trace.new_geometry,
        'condition':trace.condition,
        'IRI_average':trace.IRI_average, 
        'suspension':trace.suspension,
        'trace_type':'old_trace',
        'FilePath':os.path.relpath(trace.path, base_pth)
    })
df = pd.DataFrame(rows)
df['geom_class'] = df['geometry'].apply(type).astype(str)
df['geom_class'] = df['geom_class'].map({r"<class 'shapely.geometry.linestring.LineString'>":'Line',
                     r"<class 'shapely.geometry.multilinestring.MultiLineString'>":'MultiLine',
                    r"<class 'NoneType'>":'NoneType'})
df = df[['province','road_ID','trace_type','IRI_average','condition','suspension','geometry','geom_class','FilePath']]

#### Move Errors to Error File

In [42]:
null_geom_entires = df[['FilePath','road_ID']].loc[df.geom_class == 'NoneType']
null_geom_entires['Error'] = 'Error: Invalid geometry - no points within %s meters of eachother' % thresh
errors = pd.concat([processing_errors,null_geom_entires], axis = 0)

C:\Users\charl\Anaconda3\envs\Cfox2\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


### Section 9: Edit Road IDs for duplicates - search for link information or supply unique indexer

In [43]:
df = df.loc[df.geom_class != 'NoneType']
df2 = df.groupby('road_ID')
df['indexer'] = df2.cumcount()

duplicate_ids_dfs = []
for i in list(df2):
    if len(i[1]) == 1:
        pass
    else:
         duplicate_ids_dfs.append(i)

def check(x):
    base = x.road_ID
    search_term = x.FilePath
    search_res = re.search(r"Link.*\\", search_term)
    if search_res == None:
        return base + '_' + str(x.indexer)
    else:
        link_info = search_res.group().split('_')[0:2]
        link_info = "_".join(link_info)
        return base + '_' + link_info + str(x.indexer)
    
old, new = [], []
for i in range(0, len(duplicate_ids_dfs)):
    old.append(list(df.loc[duplicate_ids_dfs[i][1].index, 'road_ID']))
    df.loc[duplicate_ids_dfs[i][1].index, 'road_ID'] = df.loc[duplicate_ids_dfs[i][1].index].apply(lambda x: check(x), axis = 1)
    new.append(list(df.loc[duplicate_ids_dfs[i][1].index, 'road_ID']))
old = [item for sublist in old for item in sublist]
new = [item for sublist in new for item in sublist]
translations = dict(zip(old, new))

### Section 10: Save Down

In [44]:
df = df.drop(['indexer'], axis = 1)
gdf = gpd.GeoDataFrame(df, crs = {'init':'epsg:4326'}, geometry = 'geometry') 
gdf.to_file(os.path.join(r'C:\Users\charl\Documents\GOST\Vietnam\outter','roads.shp'),driver = 'ESRI Shapefile', encoding = 'utf-8')

gdf = gpd.GeoDataFrame(df, crs = {'init':'epsg:4326'}, geometry = 'geometry') 
gdf = gdf.to_crs({'init':'epsg:3405'})
gdf['length(km)'] = gdf.length / 1000
summary = gdf[['province','length(km)']].groupby('province').sum().sort_values(by = 'length(km)', ascending = False)
summary_count = gdf[['province','length(km)']].groupby('province').count()
summary['count'] = summary_count['length(km)']
summary.to_csv(os.path.join(r'C:\Users\charl\Documents\GOST\Vietnam\outter','summary_stats.csv'))

errors.to_csv(os.path.join(r'C:\Users\charl\Documents\GOST\Vietnam\outter','errors.csv'))